In [ ]:
# 실전 예제로 주식 데이터(삼성전자, 카카오, 네이버) 를 활용하여
# 주가 변동을 시각화하는 예제를 만들어보겠습니다.
#
# 이 예제에서는 Matplotlib과 Pandas를 활용하여 다음을 분석합니다:
#
# 주가 변동 라인 그래프
# 삼성전자, 카카오, 네이버의 주가 변동을 하나의 그래프로 시각화합니다.
#
# 이동 평균선 추가
# 단기(5일) 및 장기(20일) 이동 평균선을 추가하여 주가 추세를 분석합니다.
#
# 히트맵을 통한 상관관계 분석
# 삼성전자, 카카오, 네이버의 주가 상관관계를 히트맵으로 표현합니다.
#
#
# 1. 주식 데이터 수집 및 처리
# 먼저, 삼성전자(005930), 카카오(035720), 네이버(035420)의 주가 데이터를 가져오겠습니다.
# 실시간 주식 데이터를 사용하기 위해, web을 이용하여 최신 데이터를 검색합니다.
#
# 실전 예제로 삼성전자, 네이버, 카카오의 주가 데이터를 활용하여 주가 변동을 시각화하는 방법을 안내하겠습니다. 이를 통해 주가 변동 추이와 종목 간 상관관계를 분석할 수 있습니다.
#
#
#
# 1. 주가 데이터 수집
# 주가 데이터를 수집하기 위해 yahoo_fin 라이브러리를 사용합니다. 이 라이브러리는 야후 파이낸스에서 주가 데이터를 가져올 수 있도록 도와줍니다.
#
# 필요한 라이브러리 설치:
# !pip install yahoo_fin pandas matplotlib seaborn

In [2]:
# !pip install yahoo_fin pandas matplotlib seaborn
!pip install requests_html

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from yahoo_fin import stock_info as si

# 종목 코드 설정
tickers = {
    '삼성전자': '005930.KS',
    '네이버': '035420.KS',
    '카카오': '035720.KS'
}

# 데이터 수집
data = {}
for company, ticker in tickers.items():
    data[company] = si.get_data(ticker, start_date='2022-01-01', end_date='2025-01-01')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 949.2/949.2 kB 22.4 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -------------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


In [3]:
import time
import pandas as pd
import requests
from yahoo_fin import stock_info as si
import yfinance as yf

# ---------------------------
# 1. 종목 코드 설정
# ---------------------------
tickers = {
    '삼성전자': '005930.KS',
    '네이버'  : '035420.KS',
    '카카오'  : '035720.KS'
}

# 헤더 (야후 차단 방지용)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0.0.0 Safari/537.36"
    )
}

# ---------------------------
# 2. 데이터 수집 함수
# ---------------------------
def get_data_yahoo_fin(ticker, start, end, headers=None):
    return si.get_data(
        ticker,
        start_date=start,
        end_date=end,
        index_as_date=True,
        interval="1d",
        headers=headers
    )

def get_data_yfinance(ticker, start, end):
    df = yf.download(
        ticker,
        start=start,
        end=end,
        progress=False,
        auto_adjust=True,
        threads=False
    )
    if df is not None and not df.empty:
        df = df.rename(columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "adjclose",
            "Volume": "volume"
        })
    return df

def get_data_safe(ticker, start="2022-01-01", end="2025-01-01",
                  retries=3, backoff=2.0, headers=HEADERS):
    last_exc = None
    for i in range(retries):
        try:
            df = get_data_yahoo_fin(ticker, start, end, headers=headers)
            if df is not None and not df.empty:
                return df
        except Exception as e:
            last_exc = e
        time.sleep(backoff * (i + 1))
    # 폴백
    df_fb = get_data_yfinance(ticker, start, end)
    if df_fb is not None and not df_fb.empty:
        return df_fb
    raise RuntimeError(f"Failed to fetch {ticker}") from last_exc

# ---------------------------
# 3. 데이터 수집 및 CSV 저장
# ---------------------------
all_data = {}

for company, ticker in tickers.items():
    try:
        df = get_data_safe(ticker, start="2022-01-01", end="2025-09-01")
        df["company"] = company  # 회사명 컬럼 추가
        all_data[company] = df
        print(f"[OK] {company} ({ticker}) rows={len(df)}")
        time.sleep(0.7)
    except Exception as e:
        print(f"[FAIL] {company} ({ticker}) -> {e}")

# 하나의 DataFrame으로 합치기
combined = pd.concat(all_data.values())
combined.index.name = "date"

# CSV 저장
csv_path = "stock_data.csv"
combined.to_csv(csv_path, encoding="utf-8-sig")
print(f"CSV 저장 완료: {csv_path}")

# ---------------------------
# 4. 저장된 CSV 불러오기
# ---------------------------
loaded = pd.read_csv(csv_path, parse_dates=["date"])
print("불러온 데이터 크기:", loaded.shape)
print(loaded.head())


[OK] 삼성전자 (005930.KS) rows=895
[OK] 네이버 (035420.KS) rows=895
[OK] 카카오 (035720.KS) rows=895
CSV 저장 완료: stock_data.csv
불러온 데이터 크기: (2685, 9)
        date     open     high      low    close      adjclose    volume  \
0 2022-01-04  78800.0  79200.0  78300.0  78700.0  72762.867188  12427416   
1 2022-01-05  78800.0  79000.0  76400.0  77400.0  71560.953125  25470640   
2 2022-01-06  76700.0  77600.0  76600.0  76900.0  71098.671875  12931954   
3 2022-01-07  78100.0  78400.0  77400.0  78300.0  72393.070312  15163757   
4 2022-01-10  78100.0  78100.0  77100.0  78000.0  72115.687500   9947422   

      ticker company  
0  005930.KS    삼성전자  
1  005930.KS    삼성전자  
2  005930.KS    삼성전자  
3  005930.KS    삼성전자  
4  005930.KS    삼성전자  
